In [1]:
# Weather check
# Bus check
# books check
# reminders/alerts

In [3]:
# using openweathermap
from pprint import pprint
import requests
# api.openweathermap.org/data/2.5/weather?q={city name},{state},{country code}&appid={your api key}
# api.openweathermap.org/data/2.5/weather?zip=94040,us&appid={your api key}

In [125]:
class Weather:
    def __init__(self):
        # use your own API keys
        f = open("./../weatherApiKeys.txt", "r")
        self.apikey = f.read().strip()
        print("->  Getting Weather info...")
        self.req_today_weather = requests.get('http://api.openweathermap.org/data/2.5/weather?zip=94132,us&appid=' + self.apikey).json()
        print("->  Done!")
        
#         extracting and setting other variables
        self.temp_min = self.req_today_weather["main"]["temp_min"]
        self.temp_max = self.req_today_weather["main"]["temp_max"]
        self.description = self.req_today_weather["weather"][0]["description"]
        self.visibility = self.req_today_weather["visibility"] * 0.000621371 # in miles
        self.wind = self.req_today_weather["wind"]["speed"] * 2.23694 # in miles/hr
    def isWindy():
        if(self.wind > 13):
            return 1.0
        elif(self.wind > 9 and self.wind <= 13):
            return 0.5
        else:
            return 0
        def isFog():
            if(selfvisibility =< 7):
                return 1
            elif(selfvisibility =< 9 and selfvisibility > 7):
                return 0.5
            else:
                return 0
    def isCold():
        if(self.temp_max < 53 or self.temp_min <53):
            return 1
        else:
            return 0
    def getDescription():
        return self.description
    

In [110]:
(Weather().temp_min)

->  Getting Weather info...
->  Done!


283.15

In [131]:
# req_forcast_weather = requests.get('http://api.openweathermap.org/data/2.5/forecast?zip=94132,us&appid='+apiKeys)
# pprint(req_forcast_weather.json())

In [127]:
f = open("./../weatherApiKeys.txt", "r")
apikey = f.read().strip()
print("->  Getting Weather info...")
req_today_weather = requests.get('http://api.openweathermap.org/data/2.5/weather?zip=94132,us&appid=' + apikey).json()

print("->  Done!")
        
#         extracting and setting other variables
temp_min = req_today_weather["main"]["temp_min"]

temp_max = req_today_weather["main"]["temp_max"]

description = req_today_weather["weather"][0]["description"]

visibility = req_today_weather["visibility"] * 0.000621371
# miles

wind = req_today_weather["wind"]["speed"] * 2.23694
# in miles/hr




->  Getting Weather info...
->  Done!


In [129]:
description

'clear sky'

In [ ]:
def isFog():
    if(visibility =< 7):
        return 1
    elif(visibility =< 9 and visibility > 7):
        return 0.5
    else:
        return 0